# Worksheet 12

## MCS 275 Spring 2021 - David Dumas

## Topics

This worksheet focuses on **SQLite databases**.

The main references for these topics are:
* [Lecture 30 - Databases](http://dumas.io/teaching/2021/spring/mcs275/slides/lecture30.html)
* [Lecture 31 - SQL and SQLite](http://dumas.io/teaching/2021/spring/mcs275/slides/lecture31.html)
* [Lecture 32 - SQL and SQLite II](http://dumas.io/teaching/2021/spring/mcs275/slides/lecture32.html)
* Course sample code:
    * [sqlite-work notebook](https://www.dumas.io/teaching/2021/spring/mcs275/nbview/samplecode/sql/sqlite-work.html)


## Instructions

* To complete this worksheet you'll need to have matplotlib installed, or
* Problem 1 is handled differently than the others:
    * Tuesday discussion students: Problem 1 will be presented as an example at the start of discussion
    * Thursday discussion students: Please complete Problem 1 before discussion and bring your solution
* For the other problems:
    * Work on these problems in discussion.

## Star database

Questions 1 and 3 use the HYG star database in SQLite format available at:

* https://dumas.io/teaching/2021/spring/mcs275/data/hyg_data.zip

So you'll need to download this, unzip it, and put the `hyg_data.sqlite` file in a place where you can find it.

The only table in this database is called `stars`, and the columns present in that table are described at

* https://github.com/astronexus/HYG-Database

## 1. Basic star queries

Use SQL queries run against the HYG star database to answer these questions.  (You should consider both the query and its return value as part of the answer.)

### 1A. Dim stars

What are the `id`s and `proper` names of the five dimmest stars in the database (as measured by `mag`nitude)?

### 1B. Unnamed and blue

Find the positions (as `(ra,dec)` coordinates) of the ten "most blue" stars that don't have proper names in the database.  (The column `ci` is  proportional to how blue a star is.)

## 2. Programming joke utility

### Expressions in SQL

There is a new SQLite feature (or more precisely, a feature of the SQL dialect that SQLite uses for queries) that you'll need to use in this problem.

In many places where we've used column names in our queries, you can also use expressions that apply arithmetic operators and other functions to the values in the columns.  For example, if a database of MCS 275 grades has columns called `project3pct` and `project4pct`, then this query would return the email addresses of students whose grades on those two projects differed by more than 10 percent:
```
SELECT email FROM mcs275roster WHERE ABS(project3pct - project4pct) > 10;
```
You can also use expressions like this in the requested list of output columns.  For example, this query would get the average of project 3 and 4 percentages for all students, listed in alphabetical order by last name.
```
SELECT lastname, firstname, 0.5*(project3pct + project4pct) FROM mcs275roster ORDER BY lastname;
```
Such expressions can also be used after `ORDER BY` to make a custom sort.

You can find lists of built-in functions in SQLite in the documentation:
* [SQL core functions](https://sqlite.org/lang_corefunc.html)
* [SQL math functions](https://sqlite.org/lang_mathfunc.html)

### The task

Write a program that stores, delivers, and ranks programming jokes using a SQLite database.  It should support three operations:
* Add a joke: The user provides a string and the program adds it to the joke table.  In addition to the joke itself, the joke table has columns to store the number of times a joke has been told, and how many times it was successful in amusing the user.
* Tell a joke: A random joke is selected and displayed.  The user is asked if they found it amusing.  Their answer is recorded by updating the relevant counts (times told, times successful) for that row.
* Best jokes list:  Among jokes that have succeeded at least once, display the top 5 ranked by success rate (ratio of times successful to times told).

The program should create the database and table it needs if they don't already exist.  Otherwise, it should open and use the existing database.

The three functions should be selected using command line arguments.  The first command line argument is always the command---one of `add`, `tell`, or `best`.  If the command is `add`, then a second command line argument is required, which is the joke itself.  If the command is `tell`, no other arguments are required but the user is prompted for their approval/disapproval of the joke through keyboard input.

**Hints:**
1. Even without you asking to do so, SQLite will add an extra column to your table called `rowid` that has a distinct integer value for each row.  This is a *primary key*.  It won't return this column unless you ask for it explicitly (e.g. `SELECT * FROM ...` won't show it, but `SELECT rowid FROM ...` or `SELECT rowid,* FROM ...` will.  Having a unique id for each row is helpful so you can retrieve a row, and then apply an operation to the same row later.
1. SQLite doesn't have a feature to select a random row directly, but it does have a built-in function `RANDOM()` that will return a random number for each row, and you order by that.
1. In a SQLite query, if columns `A` and `B` both have type integer then `A/B` computes the *integer* division of `A` by `B`.  In contrast, `1.0*A/B` would give the true quotient because the multiplication by 1.0 converts `A` to a float (or `REAL`, in SQLite terminology).

Save the program as `jokedb.py`.

Here is a sample session of what using it should look like. (These are from a linux terminal session, where the terminal prompt is "$".  In Windows PowerShell, the prompt will look a bit different.)

```
$ python3 jokedb.py tell
ERROR: No jokes in database.

[... omitted: several jokes are added ...]

$ python3 jokedb.py tell
There are 10 types of people in the world: Those who understand binary, and those who don't.

Were you amused by this? (Y/N)y

$ python3 jokedb.py tell
The two hardest things in programming are naming things, cache invalidation, and off-by-one errors.

Were you amused by this? (Y/N)n

$ python3 jokedb.py add "Most people agree that there were no widely-used high-level programming languages before FORTRAN.  Unfortunately, there is no agreement on whether this makes FORTRAN the 1st such language, or the 0th."

$ python3 jokedb.py tell
After learning Python, my kids stopped saying 'I won't take out the garbage!'.  Instead, they say 'take_out_garbage() is deprecated in v2.0'.

Were you amused by this? (Y/N)y

$ python3 jokedb.py best
-------------------------------------------------------
#1 with 100% success rate after 8 tellings:
Knock knock.
Race condition.
Who's there?
-------------------------------------------------------
#2 with 71% success rate after 7 tellings:
There are 10 types of people in the world: Those who understand binary, and those who don't.
-------------------------------------------------------
#3 with 67% success rate after 6 tellings:
A software testing engineer walks into a bar and orders a refrigerator, -1 glasses of water, and INT_MAX+1 cans of soda.
-------------------------------------------------------
#4 with 60% success rate after 5 tellings:
After learning Python, my kids stopped saying 'I won't take out the garbage!'.  Instead, they say 'take_out_garbage() is deprecated in v2.0'.
-------------------------------------------------------
#5 with 50% success rate after 4 tellings:
The two hardest things in programming are naming things, cache invalidation, and off-by-one errors.

$
```

## 3. More complex star queries

### COUNT 

This is another SQLite feature to know about (which will be helpful in this problem).

When you make a SELECT query, if you only want to know how many rows would be returned, and not the actual data, you can ask for
```
COUNT(*)
```
in place of the list of columns you would otherwise include.  For example,
```
SELECT COUNT(*) FROM mcs275roster WHERE project4pct >= 90;
```
might return the number of students who scored 90 percent or higher on project 4.


### 3A. Siriusly bright

While `mag` is a column indicating the apparent brightness of the star as seen from earth, this is not the same as the amount of light a star emits.  Some stars seem dim only because they are very far from the earth.  To account for this, the column `absmag` (absolute magnitude) indicates the brightness of the star if it were observed from a certain standard distance (about 32 light-years).  Thus `absmag` is a measure of the light energy output of the star.  Both `mag` and `absmag` use a scale where smaller numbers correspond to more light energy.

The brightest star in the night sky is **Sirius**.  But how many stars in this database actually emit more light than Sirius (and appear dimmer due to being farther away)?

Of those, what fraction are less blue than Sirius? 

### 3B. Solar twin

What named star the database has energy output closest to that of the sun?

### 3C. Extras

Only work on these if you've finished everything else.  Solutions will not be provided.
* Suppose I see a star and want to know which one it is.  Given a position (ra,dec), an approximate color (e.g. redder than polaris, bluer than polaris, or unsure), and a viewing method (naked eye, which can see magnitudes up to about 3 near Chicago, or small telecope, which can see magnitudes up to about 8), write a Python script that will identify likely candidates from the database and report the top matches.
* Stars are moving.  The database contains velocity vectors (in x, y, z components) relative to a certain coordinate system where the sun is not moving, but only for some of the stars.  Which star in the database is moving fastest, as measured by the usual length $\| (vx,vy,vz) \| = \sqrt{vx^2 + vy^2 + vz^2}$?  How far down the list ranked by velocity (highest first) do you need to go to find a star with a name?  To find a star with magitude 3 or less?